In [1]:
import math
import numpy as np
import cmab
import env

p = 10 # total # of sensors
q = 4 # total # of misbehaved sensors
m = 5 # total # of observed sensors

tau = 50 # error ocurrance time
L = tau + 200 # length of one exp episode
N = 100 # total experiment times

# generate sigma
# some confusion on sigma[i,i]. set it to 1.0 here. not sure.
sigma = np.full((p, p), 0.5)
np.fill_diagonal(sigma, 1.0)

cmab.gamma.lamda = 0.1 # lambda

### Logging

In [2]:
def log(fname, msg):
    with open(fname, 'a+') as f:
        f.write(msg)
        f.write('\n')

### Run experiments function

In [3]:
def run_exp(delta, pattern):
    # logging file name
    fname = str(delta) + '.log'
    
    # generate mu_c
    # delta = 2.0
    mu = np.zeros((p, )) # normal mean
    mu_c = np.zeros((p, ))
    
    if pattern == 'pattern1':
        for i in range(0, int(q / 2)): # first q elements to delta, error mean
            mu_c[i*2] = delta
            mu_c[i*2+1] = delta
    elif pattern == 'pattern2':
        for i in range(0, int(q / 2)): # first q elements to delta, error mean
            mu_c[i*2] = delta
            mu_c[i*2+1] = -delta
    else:
        print('Wrong pattern!')
        return
    # print(mu_c)

    CMAB_ADD, CMAB_s_ADD, rdm_ADD, opt_ADD = [], [], [], []
    CMAB_MAX, CMAB_s_MAX, rdm_MAX, opt_MAX = [], [], [], []
    for n in range(0, N):
        Xn = env.gen_input(mu, mu_c, sigma, tau, L)
        log(fname, 'mu_c {} iter {}'.format(mu_c, n))
        print('iter', n)
        # env.visualize(Xn)

        # CMAB
        cmab.test.h = 6.0
        detect = cmab.CMAB(p, m, L, sigma, Xn, cmab.gamma)
        ADD = detect - tau
        log(fname, 'cmab {} {} {}'.format(ADD, cmab.test.max, cmab.test.h))
        if ADD > 0: CMAB_ADD.append(ADD)
        CMAB_MAX.append(cmab.test.max)
        cmab.test.max = None

        # CMAB_s
        cmab.CMAB_s.h = 1.5
        detect = cmab.CMAB_s(p, m, L, sigma, Xn, cmab.gamma)
        ADD = detect - tau
        log(fname, 'cmab_s {} {} {}'.format(ADD, cmab.CMAB_s.max, cmab.CMAB_s.h))
        if ADD > 0: CMAB_s_ADD.append(ADD)
        CMAB_s_MAX.append(cmab.CMAB_s.max)
        cmab.CMAB_s.max = None

        # random
        cmab.test.h = 6.0
        detect = cmab.rdm(p, m, L, sigma, Xn)
        ADD = detect - tau
        log(fname, 'rdm {} {} {}'.format(ADD, cmab.test.max, cmab.test.h))
        if ADD > 0: rdm_ADD.append(ADD)
        rdm_MAX.append(cmab.test.max)
        cmab.test.max = None

        # optimal, m = p
        cmab.test.h = 8.0
        detect = cmab.opt(p, L, sigma, Xn)
        ADD = detect - tau
        log(fname, 'opt {} {} {}'.format(ADD, cmab.test.max, cmab.test.h))
        if ADD > 0: opt_ADD.append(ADD)
        opt_MAX.append(cmab.test.max)
        cmab.test.max = None

    def result(label, res_list, N):
        res_list = np.array(res_list)
        res_max = np.max(res_list)
        res_min = np.min(res_list)
        res_mean = np.mean(res_list)
        res_var = np.var(res_list)
        print('\nResult of', label)
        print('Valid percentage of samples', len(res_list) / N * 100, '%')
        print('max: {} min: {} mean: {} var: {}'.format(res_max, res_min, \
                res_mean, res_var))

    result('CMAB_ADD', CMAB_ADD, N)
    result('CMAB_MAX', CMAB_MAX, N)
    result('CMAB_s_ADD', CMAB_s_ADD, N)
    result('CMAB_s_MAX', CMAB_s_MAX, N)
    result('rdm_ADD', rdm_ADD, N)
    result('rdm_MAX', rdm_MAX, N)
    result('opt_ADD', opt_ADD, N)
    result('opt_MAX', opt_MAX, N)

### Run experiment with different delta

In [4]:
run_exp(0.0, 'pattern1')

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99

Result of CMAB_ADD
Valid percentage of samples 92.0 %
max: 200 min: 200 mean: 200.0 var: 0.0

Result of CMAB_MAX
Valid percentage of samples 100.0 %
max: 9.300793625107614 min: 2.8546969880931226 mean: 4.46098

In [8]:
run_exp(1.0, 'pattern1')

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99

Result of CMAB_ADD
Valid percentage of samples 91.0 %
max: 200 min: 1 mean: 179.74725274725276 var: 2285.309745199855

Result of CMAB_MAX
Valid percentage of samples 100.0 %
max: 8.979901872239674 min: 3.91545

In [5]:
run_exp(2.0, 'pattern1')

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99

Result of CMAB_ADD
Valid percentage of samples 95.0 %
max: 58 min: 1 mean: 11.989473684210527 var: 130.66304709141272

Result of CMAB_MAX
Valid percentage of samples 100.0 %
max: 11.601193589541328 min: 6.0001

In [6]:
run_exp(0.0, 'pattern2')

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99

Result of CMAB_ADD
Valid percentage of samples 84.0 %
max: 200 min: 200 mean: 200.0 var: 0.0

Result of CMAB_MAX
Valid percentage of samples 100.0 %
max: 10.018571968421652 min: 2.788547757411111 mean: 4.73141

In [9]:
run_exp(1.0, 'pattern2')

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99

Result of CMAB_ADD
Valid percentage of samples 94.0 %
max: 200 min: 7 mean: 135.24468085106383 var: 4787.248641919421

Result of CMAB_MAX
Valid percentage of samples 100.0 %
max: 7.714395029816365 min: 4.65962

In [7]:
run_exp(2.0, 'pattern2')

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99

Result of CMAB_ADD
Valid percentage of samples 74.0 %
max: 21 min: 1 mean: 2.3513513513513513 var: 6.930606281957634

Result of CMAB_MAX
Valid percentage of samples 100.0 %
max: 16.174087924548235 min: 6.00500

In [14]:
def gamma_exp(delta, pattern):
    # generate mu_c
    # delta = 2.0
    mu = np.zeros((p, )) # normal mean
    mu_c = np.zeros((p, ))
    
    if pattern == 'pattern1':
        for i in range(0, int(q / 2)): # first q elements to delta, error mean
            mu_c[i*2] = delta
            mu_c[i*2+1] = delta
    elif pattern == 'pattern2':
        for i in range(0, int(q / 2)): # first q elements to delta, error mean
            mu_c[i*2] = delta
            mu_c[i*2+1] = -delta
    else:
        print('Wrong pattern!')
        return
    # print(mu_c)

    CMAB_ADD = []
    CMAB_MAX = []
    for n in range(0, N):
        Xn = env.gen_input(mu, mu_c, sigma, tau, L)
        print('iter', n)
        # env.visualize(Xn)

        # CMAB
        cmab.test.h = 6.0
        detect = cmab.CMAB(p, m, L, sigma, Xn, cmab.gamma)
        ADD = detect - tau
        if ADD > 0: CMAB_ADD.append(ADD)
        CMAB_MAX.append(cmab.test.max)
        cmab.test.max = None


    def result(label, res_list, N):
        res_list = np.array(res_list)
        res_max = np.max(res_list)
        res_min = np.min(res_list)
        res_mean = np.mean(res_list)
        res_var = np.var(res_list)
        print('\nResult of', label)
        print('Valid percentage of samples', len(res_list) / N * 100, '%')
        print('max: {} min: {} mean: {} var: {}'.format(res_max, res_min, \
                res_mean, res_var))

    result('CMAB_ADD', CMAB_ADD, N)
    result('CMAB_MAX', CMAB_MAX, N)

In [17]:
cmab.gamma.lamda = 0.01 # lambda
N = 10
gamma_exp(0.0, 'pattern2')
gamma_exp(2.0, 'pattern2')

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9

Result of CMAB_ADD
Valid percentage of samples 70.0 %
max: 200 min: 200 mean: 200.0 var: 0.0

Result of CMAB_MAX
Valid percentage of samples 100.0 %
max: 6.9344945022661335 min: 2.411854847886132 mean: 4.516572108794659 var: 2.6476185509310652
iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9

Result of CMAB_ADD
Valid percentage of samples 80.0 %
max: 200 min: 200 mean: 200.0 var: 0.0

Result of CMAB_MAX
Valid percentage of samples 100.0 %
max: 6.6495085722652245 min: 2.004940411733182 mean: 3.987260829307872 var: 2.7224452666335397


In [18]:
cmab.gamma.lamda = 0.05 # lambda
N = 10
gamma_exp(0.0, 'pattern2')
gamma_exp(2.0, 'pattern2')

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9

Result of CMAB_ADD
Valid percentage of samples 100.0 %
max: 200 min: 200 mean: 200.0 var: 0.0

Result of CMAB_MAX
Valid percentage of samples 100.0 %
max: 5.860391484248318 min: 1.9559830808379033 mean: 3.887884666132153 var: 1.3282589734121442
iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9

Result of CMAB_ADD
Valid percentage of samples 100.0 %
max: 200 min: 39 mean: 115.2 var: 4231.960000000001

Result of CMAB_MAX
Valid percentage of samples 100.0 %
max: 7.413699499735191 min: 5.3860505728625645 mean: 6.210966933823428 var: 0.3419775328456645


In [19]:
cmab.gamma.lamda = 0.1 # lambda
gamma_exp(0.0, 'pattern2')
gamma_exp(2.0, 'pattern2')

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9

Result of CMAB_ADD
Valid percentage of samples 100.0 %
max: 200 min: 200 mean: 200.0 var: 0.0

Result of CMAB_MAX
Valid percentage of samples 100.0 %
max: 5.588937838803387 min: 3.1866439728163134 mean: 4.234947746507601 var: 0.4647661396586016
iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9

Result of CMAB_ADD
Valid percentage of samples 100.0 %
max: 11 min: 1 mean: 3.8 var: 13.76

Result of CMAB_MAX
Valid percentage of samples 100.0 %
max: 9.503411516682286 min: 6.095228180623232 mean: 7.548233204089422 var: 1.1426124802941453
